In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git@nightly git+https://github.com/unslothai/unsloth-zoo.git

In [9]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.11.6 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

In [11]:
dataset = dataset.train_test_split(
    test_size=0.05,
    seed=3407
)

train_dataset = dataset["train"]
eval_dataset  = dataset["test"]

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
train_dataset = standardize_sharegpt(train_dataset)
train_dataset = train_dataset.map(formatting_prompts_func, batched = True,)
eval_dataset  = standardize_sharegpt(eval_dataset)
eval_dataset  = eval_dataset.map(formatting_prompts_func, batched = True,)


We look at how the conversations are structured for item 5:

In [ ]:
dataset[5]["conversations"]

In [ ]:
train_dataset[5]["conversations"]

In [ ]:
eval_dataset[5]["conversations"]

[{'content': 'Explain how the principle of non-contradiction can be applied to prove that any prime number greater than 2 cannot be expressed as the sum of two smaller prime numbers, and use this principle to demonstrate the truth or falsity of the given statement.',
  'role': 'user'},
 {'content': "The principle of non-contradiction states that a statement and its negation cannot both be true at the same time. \nTo prove that any prime number greater than 2 cannot be expressed as the sum of two smaller prime numbers, we can assume the opposite and show that it leads to a contradiction. \nLet's assume that there exists a prime number p greater than 2 that can be expressed as the sum of two smaller prime numbers, say p = q + r, where q and r are smaller prime numbers. \nSince q and r are smaller than p, they cannot be equal to p. Therefore, they must be less than p. \nBut since p is a prime number, it can only be divided by 1 and itself. Therefore, neither q nor r can divide p. \nNow, l

And we see how the chat template transformed these conversations.

**[Notice]** Llama 3.1 Instruct's default chat template default adds `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, so do not be alarmed!

In [ ]:
dataset[5]["text"]

In [ ]:
train_dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is a Python code snippet that can be used to convert a given Fahrenheit temperature to Celsius?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nYou can use the following Python code snippet to convert Fahrenheit temperature to Celsius:\n\n```python\ndef fahrenheit_to_celsius(fahrenheit):\n    celsius = (fahrenheit - 32) * (5 / 9)\n    return celsius\n```\n\nExplanation:\nIn this code snippet, we define a function called `fahrenheit_to_celsius`, which takes a parameter `fahrenheit` representing the temperature in Fahrenheit. Inside the function, we calculate the equivalent temperature in Celsius using the formula `(fahrenheit - 32) * (5 / 9)`. This formula converts Fahrenheit to Celsius by subtracting 32 from the Fahrenheit temperature and then multiplying the result by 5/9.\n\nFinally, the

In [ ]:
eval_dataset[5]["text"]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nExplain how the principle of non-contradiction can be applied to prove that any prime number greater than 2 cannot be expressed as the sum of two smaller prime numbers, and use this principle to demonstrate the truth or falsity of the given statement.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe principle of non-contradiction states that a statement and its negation cannot both be true at the same time. \nTo prove that any prime number greater than 2 cannot be expressed as the sum of two smaller prime numbers, we can assume the opposite and show that it leads to a contradiction. \nLet's assume that there exists a prime number p greater than 2 that can be expressed as the sum of two smaller prime numbers, say p = q + r, where q and r are smaller prime numbers. \nSince q and r are smaller 

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 250,
        save_steps = 50,
        save_total_limit = 3,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        # output_dir = "outputs",
        output_dir = "/content/drive/MyDrive/LLM_fineTuning/outputs_1B_train",
        report_to = "none", # Use this for WandB etc
    ),
)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

We verify masking is actually done:

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is a Python code snippet that can be used to convert a given Fahrenheit temperature to Celsius?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nYou can use the following Python code snippet to convert Fahrenheit temperature to Celsius:\n\n```python\ndef fahrenheit_to_celsius(fahrenheit):\n    celsius = (fahrenheit - 32) * (5 / 9)\n    return celsius\n```\n\nExplanation:\nIn this code snippet, we define a function called `fahrenheit_to_celsius`, which takes a parameter `fahrenheit` representing the temperature in Fahrenheit. Inside the function, we calculate the equivalent temperature in Celsius using the formula `(fahrenheit - 32) * (5 / 9)`. This formula converts Fahrenheit to Celsius by subtracting 32 from the Fahrenheit temperature and then multiplying the result by 5/9

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                       You can use the following Python code snippet to convert Fahrenheit temperature to Celsius:\n\n```python\ndef fahrenheit_to_celsius(fahrenheit):\n    celsius = (fahrenheit - 32) * (5 / 9)\n    return celsius\n```\n\nExplanation:\nIn this code snippet, we define a function called `fahrenheit_to_celsius`, which takes a parameter `fahrenheit` representing the temperature in Fahrenheit. Inside the function, we calculate the equivalent temperature in Celsius using the formula `(fahrenheit - 32) * (5 / 9)`. This formula converts Fahrenheit to Celsius by subtracting 32 from the Fahrenheit temperature and then multiplying the result by 5/9.\n\nFinally, the function returns the calculated temperature in Celsius. To use this code snippet, you can call the `fahrenheit_to_celsius` function and provide the Fahrenheit temperature as an argument. The function will then return the equivalent temperature in Celsius.\n\nExample usage:\n\n```python\

We can see the System and Instruction prompts are successfully masked!

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
4.033 GB of memory reserved.


In [ ]:
# trainer_stats = trainer.train(resume_from_checkpoint=True)
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 95,000 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)


Step,Training Loss
1,0.865900
2,0.698300
3,0.977600
4,0.761500
5,0.906300
6,0.821400
7,1.096100
8,0.817200
9,1.031200
10,0.872000


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1475.8207 seconds used for training.
24.6 minutes used for training.
Peak reserved memory = 4.094 GB.
Peak reserved memory for training = 0.061 GB.
Peak reserved memory % of max memory = 27.773 %.
Peak reserved memory for training % of max memory = 0.414 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContinue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe given sequence is a Fibonacci sequence. \nTo continue the sequence, we need to identify the next number based on the rule of the Fibonacci sequence.\n1. If a Fibonacci number is an even number, it is added to the list as it is.\n2. If a Fibonacci number is an odd number, its successor']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17715, 28657, 46368, 75025, 121393, 196418, 317811, 518289, 835377, 1346709, 2175767, 3566697, 5831056, 9666720,


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("/content/drive/MyDrive/LLM_fineTuning/lora_model_1B_train") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/LLM_fineTuning/lora_model_1B_train")
model.push_to_hub("Alyssa12375/lora_model_1B_train", token = "Y") # Online saving
tokenizer.push_to_hub("Alyssa12375/lora_model_1B_train", token = "") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/LLM_fineTuning/lora_model_1B_train", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    # FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# messages = [
#     {"role": "user", "content": "Describe a tall tower in the capital of France."},
# ]
# inputs = tokenizer.apply_chat_template(
#     messages,
#     tokenize = True,
#     add_generation_prompt = True, # Must add for generation
#     return_tensors = "pt",
# ).to("cuda")

# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer, skip_prompt = True)
# _ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
#                    use_cache = True, temperature = 1.5, min_p = 0.1)

==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if True: model.save_pretrained_merged("/content/drive/MyDrive/LLM_fineTuning/model_1B_train", tokenizer, save_method = "merged_16bit",)
if True: model.push_to_hub_merged("Alyssa12375/model_1B_train", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp

!pip install -r llama.cpp/requirements.txt

!make -C llama.cpp
# !pip install --upgrade transformers huggingface_hub sentencepiece

fatal: destination path 'llama.cpp' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
make: Entering directory '/content/llama.cpp'
Makefile:6: *** Build system changed:
 The Makefile build has been replaced by CMake.

 For build instructions see:
 https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md

.  Stop.
make: Leaving directory '/content/llama.cpp'


In [ ]:
import json
import os

model_path = "/content/drive/MyDrive/LLM_fineTuning/model_1B_train/"
config_file = os.path.join(model_path, "tokenizer_config.json")

if os.path.exists(config_file):
    with open(config_file, "r") as f:
        config = json.load(f)

    config["tokenizer_class"] = "PreTrainedTokenizerFast"

    if "legacy" in config:
        del config["legacy"]

    with open(config_file, "w") as f:
        json.dump(config, f, indent=4)

else:
    print("error")

In [ ]:
!python llama.cpp/convert_hf_to_gguf.py \
    /content/drive/MyDrive/LLM_fineTuning/model_1B_train \
    --outfile temp_fp16.gguf \
    --outtype f16


INFO:hf-to-gguf:Loading model: model_1B_train
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:hf-to-gguf:gguf: indexing model part 'model.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {32}
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> F16, shape = {2048, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.bfloat16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.bfloat16 --> F16, shape = {8192, 2048}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.bfloat16 --> F16, shape = {2048, 8192}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.bfloat16 --> F16, shape = {2048, 8192}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.bfloat16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.bfloat16 --> F16, shape = {2048, 512}
INFO:hf-to-gguf:blk.0.attn_output.w

In [ ]:
import os
import sys

!pip install cmake --upgrade > /dev/null

!cd /content/llama.cpp && cmake -B build && cmake --build build --config Release -j 4


tool_path = "/content/llama.cpp/build/bin/llama-quantize"

if not os.path.exists(tool_path):
    tool_path = "/content/llama.cpp/build/llama-quantize"

if not os.path.exists(tool_path):
    sys.exit()


source_file = "/content/temp_fp16.gguf"
output_file = "/content/drive/MyDrive/LLM_fineTuning/model_1B_train_q4_k_m.gguf"


!{tool_path} {source_file} {output_file} q4_k_m

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.34.1")
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- GGML_SYSTEM_ARCH: x86
-- Including CPU backend
-- Found OpenMP_C: 

In [ ]:
# # Save to 8bit Q8_0
# if False: model.save_pretrained_gguf("model", tokenizer,)
# # Remember to go to https://huggingface.co/settings/tokens for a token!
# # And change hf to your username!
# if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# # Save to 16bit GGUF
# if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
# if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# # Save to q4_k_m GGUF
# if False: model.save_pretrained_gguf("/content/drive/MyDrive/LLM_fineTuning/GGUFmodel", tokenizer, quantization_method = "q4_k_m")
# if True: model.push_to_hub_gguf("Alyssa12375/GGUFmodel_1B", tokenizer, quantization_method = "q4_k_m", token = "")

# # Save to multiple GGUF options - much faster if you want multiple!
# if False:
#     model.push_to_hub_gguf(
#         "hf/model", # Change hf to your username!
#         tokenizer,
#         quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
#         token = "", # Get a token at https://huggingface.co/settings/tokens
    )

In [ ]:
from huggingface_hub import HfApi, create_repo

token = ""
repo_id = "Alyssa12375/GGUFmodel_1B_train"

local_file_path = "/content/drive/MyDrive/LLM_fineTuning/model_1B_train_q4_k_m.gguf"

path_in_repo = "model_1B_train_q4_k_m.gguf"
api = HfApi(token=token)
create_repo(repo_id, repo_type="model", exist_ok=True, token=token)

api.upload_file(
    path_or_fileobj=local_file_path,
    path_in_repo=path_in_repo,
    repo_id=repo_id,
    repo_type="model",
    token=token
)


Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)
10. [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
11. [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
12. [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>

In [ ]:
import random
import json
import torch
from datasets import load_dataset
from unsloth import FastLanguageModel

# =====================================================
# setting up
# =====================================================

# Base & Fine-tuned models (Unsloth / HF)
BASE_MODEL_NAME = "unsloth/Llama-3.2-1B-Instruct"
FT_MODEL_PATH   = "Alyssa12375/model_1B"

# Judge model (Unsloth Qwen)
JUDGE_MODEL_NAME = "unsloth/Qwen2.5-7B-Instruct-bnb-4bit"

NUM_EVAL_SAMPLES = 30
MAX_GEN_TOKENS   = 256
MAX_JUDGE_TOKENS = 64
MAX_SEQ_LENGTH   = 2048
dtype = None

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# =====================================================
# load eval dataset
# =====================================================

# dataset = load_dataset("mlabonne/FineTome-100k", split="train")
# dataset = dataset.train_test_split(test_size=0.05, seed=42)
# eval_dataset = dataset["test"]

# =====================================================
# load base model
# =====================================================

print("Loading base model...")
base_model, base_tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype = dtype,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(base_model)

# =====================================================
# load fine-tuned model
# =====================================================

print("Loading fine-tuned model...")
ft_model, ft_tokenizer = FastLanguageModel.from_pretrained(
    model_name=FT_MODEL_PATH,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype = dtype,
    load_in_4bit=True,
)

# =====================================================
# load judge model
# =====================================================

print("Loading Qwen judge model...")
judge_model, judge_tokenizer = FastLanguageModel.from_pretrained(
    model_name=JUDGE_MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype = dtype,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(judge_model)

In [17]:
from unsloth.chat_templates import get_chat_template

# =====================================================
# useful function
# =====================================================

def extract_user_prompt(example):
    for turn in example["conversations"]:
        if turn["role"] == "user":
            return turn["content"]
    return None

def generate(model, tokenizer, prompt, max_new_tokens=256):
    messages = [
        {"role": "user", "content": prompt}
    ]

    tokenizer = get_chat_template(
        tokenizer,
        chat_template="llama-3.1",
    )

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(DEVICE)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            use_cache=True,
        )

    generated = outputs[0][inputs.shape[-1]:]
    return tokenizer.decode(
        generated, skip_special_tokens=True
    ).strip()

def build_judge_messages(user_prompt, A, B):
    return [
        {
            "role": "system",
            "content": (
                "You are a strict judge.\n"
                "You must choose which answer is better.\n"
                "Reply with ONLY ONE letter:\n"
                "A (Answer A), B (Answer B), or T (Tie).\n"
                "Do not explain."
            ),
        },
        {
            "role": "user",
            "content": (
                f"Instruction:\n{user_prompt}\n\n"
                f"Answer A:\n{A}\n\n"
                f"Answer B:\n{B}\n\n"
                "Which answer is better?\n"
                "Reply with ONLY ONE letter."
            ),
        },
    ]


def judge_with_qwen(judge_messages):
    inputs = judge_tokenizer.apply_chat_template(
        judge_messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(DEVICE)

    with torch.no_grad():
        outputs = judge_model.generate(
            input_ids=inputs,
            max_new_tokens=1,
            do_sample=False,
            temperature=0.0,
            use_cache=True,
        )

    decoded = judge_tokenizer.decode(
        outputs[0], skip_special_tokens=True
    ).strip()

    return decoded[-1]


# =====================================================
# Collect eval prompts
# =====================================================

eval_prompts = []
for ex in eval_dataset:
    p = extract_user_prompt(ex)
    if p:
        eval_prompts.append(p)
    if len(eval_prompts) >= NUM_EVAL_SAMPLES:
        break

print(f"Collected {len(eval_prompts)} evaluation prompts.")

# =====================================================
# Eval loop
# =====================================================

results = []

for idx, prompt in enumerate(eval_prompts):
    print(f"Evaluating {idx+1}/{len(eval_prompts)}")

    base_out = generate(base_model, base_tokenizer, prompt, MAX_GEN_TOKENS)
    ft_out   = generate(ft_model, ft_tokenizer, prompt, MAX_GEN_TOKENS)

    if random.random() < 0.5:
        A, B = base_out, ft_out
        mapping = {"A": "base", "B": "finetuned"}
    else:
        A, B = ft_out, base_out
        mapping = {"A": "finetuned", "B": "base"}

    judge_messages = build_judge_messages(prompt, A, B)
    judge_choice = judge_with_qwen(judge_messages)

    if judge_choice == "A":
        winner = "base"
    elif judge_choice == "B":
        winner = "finetuned"
    else:
        winner = "tie"


    results.append({
        "prompt": prompt,
        "base_output": base_out,
        "finetuned_output": ft_out,
        "winner": winner,
        "judge_output": judge_choice
    })

# =====================================================
# save the results
# =====================================================

with open("eval_results_unsloth_only.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print("Evaluation complete (Unsloth-only).")


Collected 30 evaluation prompts.
Evaluating 1/30
Evaluating 2/30
Evaluating 3/30
Evaluating 4/30
Evaluating 5/30
Evaluating 6/30
Evaluating 7/30
Evaluating 8/30
Evaluating 9/30
Evaluating 10/30
Evaluating 11/30
Evaluating 12/30
Evaluating 13/30
Evaluating 14/30
Evaluating 15/30
Evaluating 16/30
Evaluating 17/30
Evaluating 18/30
Evaluating 19/30
Evaluating 20/30
Evaluating 21/30
Evaluating 22/30
Evaluating 23/30
Evaluating 24/30
Evaluating 25/30
Evaluating 26/30
Evaluating 27/30
Evaluating 28/30
Evaluating 29/30
Evaluating 30/30
Evaluation complete (Unsloth-only).


In [18]:
import json
import pandas as pd
from collections import Counter

JSON_PATH = "/content/eval_results_unsloth_only.json"

with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

counter = Counter(x["winner"] for x in data)

base_win = counter.get("base", 0)
ft_win   = counter.get("finetuned", 0)
tie      = counter.get("tie", 0)
total    = len(data)

denom = max(1, base_win + ft_win)

df = pd.DataFrame([{
    "total_samples": total,
    "base_wins": base_win,
    "finetuned_wins": ft_win,
    "ties": tie,
    "finetuned_win_rate(excl_tie)": round(ft_win / denom, 3),
    "base_win_rate(excl_tie)": round(base_win / denom, 3),
}])

print(df)
df.to_csv("eval_summary.csv", index=False)


   total_samples  base_wins  finetuned_wins  ties  \
0             30         10              20     0   

   finetuned_win_rate(excl_tie)  base_win_rate(excl_tie)  
0                         0.667                    0.333  
